# Finetuning a Convolutional Neural Network

In this exercise, you will have to finetune a pretrained CNN model on the CIFAR10 dataset. The data loading and model testing logic are already included in your code. You will have to create the model and the training loop.

**In this workspace you have GPU to help train the model but it is best practice to DISABLE it while writing code and only ENABLE it when you are training.** 

Here are the steps you need to do to complete this exercise:

1. Finish the `create_model()` function. You should use a pretrained model. You are free to choose any pre-trained model that you want to use. 
2. Finish the `train()` function. This function should validate the accuracy of the model during the training stage. You should stop the training when this validation accuracy stops increasing.
3. Save all your work and then **ENABLE** the GPU
4. Run the file to make sure that the model is training properly.
5. If it works, remember to **DISABLE** the GPU before moving to the next page. 

In case you get stuck, you can look at the solution by clicking the jupyter symbol at the top left and navigating to `finetune_a_cnn_solution.py`.

## Try It Out!
- See how your accuracy changes when using other pre-trained models.
- Play around with the number of layers and neurons in your model. How does the accuracy change? How long does it take to train the model?
- Can you create the same network in TensorFlow as well?

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

def test(model, test_loader, criterion, device):
    model.eval()
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        inputs=inputs.to(device)
        labels=labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects / len(test_loader.dataset)
    
    print(f"Testing Accuracy: {100*total_acc}, Testing Loss: {total_loss}")
    
def train(model, train_loader, validation_loader, criterion, optimizer, device):
    epochs=50
    best_loss=1e6
    image_dataset={'train':train_loader, 'valid':validation_loader}
    loss_counter=0
    
    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase=='train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            running_samples=0

            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                inputs=inputs.to(device)
                labels=labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase=='train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                running_samples+=len(inputs)
                if running_samples % 2000  == 0:
                    accuracy = running_corrects/running_samples
                    print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%)".format(
                            running_samples,
                            len(image_dataset[phase].dataset),
                            100.0 * (running_samples / len(image_dataset[phase].dataset)),
                            loss.item(),
                            running_corrects,
                            running_samples,
                            100.0*accuracy,
                        )
                    )
                
                #NOTE: Comment lines below to train and test on whole dataset
                if running_samples>(0.2*len(image_dataset[phase].dataset)):
                    break

            epoch_loss = running_loss / running_samples
            epoch_acc = running_corrects / running_samples
            
            if phase=='valid':
                if epoch_loss<best_loss:
                    best_loss=epoch_loss
                else:
                    loss_counter+=1

        if loss_counter==1:
            break
        #TODO: Finish the rest of the training code
        # The code should stop training when the validation accuracy
        # stops increasing
    return model

def create_model():
    # TODO: Write code to create the model
    model = models.resnet18(pretrained=True)
    #model = models.resnet34(pretrained=True)
    #model = models.squeezenet1_0(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False   

    num_features=model.fc.in_features
    model.fc = nn.Sequential(
                   nn.Linear(num_features, 10))
    #model.fc = nn.Sequential(nn.Linear(num_features, 100), nn.Linear(100,10))
    return model

batch_size=10
training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

testing_transform = transforms.Compose([transforms.ToTensor(),
    transforms.RandomResizedCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on Device {device}")

model=create_model()
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

train(model, trainloader, testloader, criterion, optimizer, device)

test(model, testloader, criterion, device)

Files already downloaded and verified
Files already downloaded and verified
Running on Device cpu


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:01<00:00, 28411148.42it/s]


Epoch 0, Phase train
Images [2000/50000 (4%)] Loss: 1.73 Accuracy: 889/2000 (44.45%)
Images [4000/50000 (8%)] Loss: 0.87 Accuracy: 2107/4000 (52.68%)
Images [6000/50000 (12%)] Loss: 1.18 Accuracy: 3453/6000 (57.55%)
Images [8000/50000 (16%)] Loss: 0.89 Accuracy: 4808/8000 (60.10%)
Images [10000/50000 (20%)] Loss: 0.45 Accuracy: 6150/10000 (61.50%)
Epoch 0, Phase valid


TypeError: 'builtin_function_or_method' object is not subscriptable

## Remember to Disable GPU when you are done training. 